In [1]:
import json
import pandas as pd
import time
import datetime
from collections import defaultdict
from sql_queries import events_table_insert, teams_table_insert, players_table_insert, related_events_table_insert, matches_table_insert, referees_table_insert
from create_tables import create_database, drop_tables, create_tables
from collections import Counter

In [2]:
def find_common_keys(json_dicts):
    high_level_keys = Counter()
    dict_count = len(json_dicts)
    for d in json_dicts:
        high_level_keys.update(d.keys())
    print(high_level_keys.keys())
    # identify columns appearing in at least half of data dictionaries
    common_keys = [key for key in high_level_keys.keys() if high_level_keys[key] == dict_count]
    
    return common_keys

In [3]:
with open('./data/15946.json', 'r+') as f:
    match_events = json.load(f)

In [4]:
common_keys = find_common_keys(match_events)

dict_keys(['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type', 'possession', 'possession_team', 'play_pattern', 'team', 'duration', 'tactics', 'related_events', 'player', 'position', 'location', 'pass', 'carry', 'ball_receipt', 'counterpress', 'under_pressure', 'ball_recovery', 'shot', 'goalkeeper', 'duel', 'dribble', 'foul_committed', 'interception', 'block', 'foul_won', 'bad_behaviour', 'substitution'])


In [5]:
common_keys

['id',
 'index',
 'period',
 'timestamp',
 'minute',
 'second',
 'type',
 'possession',
 'possession_team',
 'play_pattern',
 'team']

In [6]:
cur, conn = create_database()

drop_tables(cur, conn)
create_tables(cur, conn)

In [7]:
def events_table_upload(cur, row_dict):
    cur.execute(
        events_table_insert,
        (
            row_dict["id"],
            row_dict["index"],
            row_dict["type"],
            row_dict["play_pattern"],
            row_dict["event_time"],
            row_dict["period"],
            row_dict["location_x"],
            row_dict["location_y"],
            row_dict["possession_team"],
            row_dict["possession"],
            row_dict["team"],
            row_dict["player"],
            row_dict["team"],
            row_dict["duration"],
        )
    )

In [8]:
def teams_table_upload(cur, row_dict):
    cur.execute(
        teams_table_insert,
        (
            row_dict["id"],
            row_dict["name"],
            row_dict["gender"],
            row_dict["country"]
        )
    )

In [9]:
def players_table_upload(cur, row_dict):
    if row_dict["id"]:
        cur.execute(
            players_table_insert,
            (
                row_dict["id"],
                row_dict["name"],
                row_dict["position"],
                row_dict["team"]
            )
        )

In [ ]:
def referees_table_upload(cur, row_dict):
    if row_dict["id"]:
        cur.execute(
            referees_table_insert,
            (
                row_dict["id"],
                row_dict["name"],
                row_dict["country"]
            )
        )

In [ ]:
def matches_table_upload(cur, row_dict):
    cur.execute(
        matches_table_insert,
        (
            row_dict["id"],
            row_dict["date"],
            row_dict["kick_off"],
            row_dict["competition"],
            row_dict["season"],
            row_dict["match_week"],
            row_dict["competition_stage"],
            row_dict["stadium"],
            row_dict["home_team"],
            row_dict["away_team"],
            row_dict["home_score"],
            row_dict["away_score"],
            row_dict["match_status"],
            row_dict["last_updated"]
        )
    )

In [10]:
def def_value():
    return None

In [11]:
for event in match_events:
    events_table_dict = defaultdict(def_value)
    teams_table_dict = defaultdict(def_value)
    players_table_dict = defaultdict(def_value)
    
    events_table_dict["id"] = event["id"]
    events_table_dict["index"] = event["index"]
    events_table_dict["type"] = event["type"]["name"]
    events_table_dict["play_pattern"] = event["play_pattern"]["name"]
    events_table_dict["event_time"] = event["timestamp"]
    events_table_dict["period"] = event["period"]
    events_table_dict["possession"] = event["possession"]
    events_table_dict["possession_team"] = event["possession_team"]["id"]
    events_table_dict["team"] = event["team"]["id"]
    
    teams_table_dict["id"] = event["team"]["id"]
    teams_table_dict["name"] = event["team"]["name"]
    
    players_table_dict["team"] = event["team"]["id"]
    
    
    # TODO: assign the match ID to the events_table_dict and matches_table_dict
    events_table_dict["match"] = 15946
    
    for key, value in event.items():
        if key == "location":
            location_x, location_y = value
            
            events_table_dict["location_x"] = location_x,
            events_table_dict["location_y"] = location_y
            
        elif key == "possession_team":
            events_table_dict["possession_team"] = value["id"]
            
        elif key == "player":
            events_table_dict["player"] = value["id"]
            
            players_table_dict["id"] = value["id"]
            players_table_dict["name"] = value["name"]
            
        elif key == "duration":
            events_table_dict["duration"] = value
            
        elif key == "position":
            players_table_dict["position"] = value["name"]
            
        elif key == "related_events":
            for event_id in value:
                cur.execute(
                    related_events_table_insert,
                    (
                        event["id"],
                        event_id
                    )
                )
            
#     cur.execute(
#         events_table_insert,
#         (
#             events_table_dict["id"],
#             events_table_dict["index"],
#             events_table_dict["type"],
#             events_table_dict["play_pattern"],
#             events_table_dict["event_time"],
#             events_table_dict["period"],
#             events_table_dict["location_x"],
#             events_table_dict["location_y"],
#             events_table_dict["possession_team"],
#             events_table_dict["possession"],
#             events_table_dict["team"],
#             events_table_dict["player"],
#             events_table_dict["team"],
#             events_table_dict["duration"],
#         )
#     )
    events_table_upload(cur, events_table_dict)
    
#     cur.execute(
#         teams_table_insert,
#         (
#             teams_table_dict["id"],
#             teams_table_dict["name"],
#             teams_table_dict["gender"],
#             teams_table_dict["country"]
#         )
#     )
    teams_table_upload(cur, teams_table_dict)
    
#     if players_table_dict["id"]:
#         cur.execute(
#             players_table_insert,
#             (
#                 players_table_dict["id"],
#                 players_table_dict["name"],
#                 players_table_dict["position"],
#                 players_table_dict["team"]
#             )
#         )
    players_table_upload(cur, players_table_dict)
conn.commit()

In [12]:
with open('../open-data/data/matches/11/1.json', 'r+') as f:
    match_details = json.load(f)

In [13]:
match_keys = find_common_keys(match_details)

dict_keys(['match_id', 'match_date', 'kick_off', 'competition', 'season', 'home_team', 'away_team', 'home_score', 'away_score', 'match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'metadata', 'match_week', 'competition_stage', 'stadium', 'referee'])


In [14]:
match_keys

['match_id',
 'match_date',
 'kick_off',
 'competition',
 'season',
 'home_team',
 'away_team',
 'home_score',
 'away_score',
 'match_status',
 'match_status_360',
 'last_updated',
 'last_updated_360',
 'metadata',
 'match_week',
 'competition_stage',
 'stadium']

In [15]:
for match in match_details:
    match_table_dict = defaultdict(def_value)
    referees_table_dict = defaultdict(def_value)
    
    match_table_dict["id"] = match["match_id"],
    match_table_dict["date"] = match["match_date"],
    match_table_dict["kick_off"] = match["kick_off"],
    match_table_dict["competition"] = match["competition"]["competition_id"],
    match_table_dict["season"] = match["season"]["season_name"],
    match_table_dict["match_week"] = match["match_week"],
    match_table_dict["competition_stage"] = match["competition_stage"]["id"],
    match_table_dict["stadium"] = match["stadium"]["id"],
    match_table_dict["home_team"] = match["home_team"]["home_team_id"],
    match_table_dict["away_team"] = match["away_team"]["away_team_id"],
    match_table_dict["home_score"] = match["home_score"],
    match_table_dict["away_score"] = match["away_score"],
    match_table_dict["match_status"] = match["match_status"],
    match_table_dict["last_updated"] = match["last_updated"]
    
    for key, value in match.items():
        if key == "referee":
            referees_table_dict["id"] = value["id"]
            referees_table_dict["name"] = value["name"]
            referees_table_dict["country"] = value["country"]["id"]    

#     cur.execute(
#         matches_table_insert,
#         (
#             match_table_dict["id"],
#             match_table_dict["date"],
#             match_table_dict["kick_off"],
#             match_table_dict["competition"],
#             match_table_dict["season"],
#             match_table_dict["match_week"],
#             match_table_dict["competition_stage"],
#             match_table_dict["stadium"],
#             match_table_dict["home_team"],
#             match_table_dict["away_team"],
#             match_table_dict["home_score"],
#             match_table_dict["away_score"],
#             match_table_dict["match_status"],
#             match_table_dict["last_updated"]
#         )
#     )
    matches_table_upload(cur, match_table_dict)
    
#     if referees_table_dict["id"]:
#         cur.execute(
#             referees_table_insert,
#             (
#                 referees_table_dict["id"],
#                 referees_table_dict["name"],
#                 referees_table_dict["country"]
#             )
#         )
    referees_table_upload(cur, referees_table_dict)
conn.commit()

In [16]:
conn.close()